In [1]:
from google.adk.tools.bigquery import BigQueryCredentialsConfig, BigQueryToolset
from google.adk.tools.bigquery.config import BigQueryToolConfig, WriteMode
from google.cloud import bigquery

from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.runners import InMemoryRunner, Runner
from google.adk.models.google_llm import Gemini
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.tools import AgentTool
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import preload_memory

import os, google.auth, warnings, contextlib, io

In [2]:
PROJECT_ID = os.environ["PROJECT_ID"]
DATASET_ID = os.environ["DATASET_ID"]
TABLE_ID = os.environ["TABLE_ID2"]
BQ_CRED_ID = os.environ["BQ_CRED_PATH"]
APP_NAME = APP_NAME = "MemoryDemoApp"
USER_ID = "demo_user"
SCOPES = ["https://www.googleapis.com/auth/bigquery"]

client = bigquery.Client()
warnings.filterwarnings("ignore", category=UserWarning, message=r"\[EXPERIMENTAL\]")
supp_warning = io.StringIO()

In [3]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
    )

async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )
    
tool_config = BigQueryToolConfig(write_mode=WriteMode.BLOCKED)
creds, _ = google.auth.load_credentials_from_file(BQ_CRED_ID)
credentials_config = BigQueryCredentialsConfig(credentials=creds)
bigquery_toolset = BigQueryToolset(
    credentials_config=credentials_config,   
    tool_filter=['list_dataset_ids', 'get_dataset_info', 'list_table_ids', 'get_table_info', 'execute_sql'],
    )

summarizer_agent = LlmAgent(
    name="SummarizerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
        You are a summarization agent.
        
        **You receive**:
            - rows: A list of SQL query results.
        
        **Your task**:
            1. Produce a text summary with 1 to 2 paragraphs and a 3–5 bullet summary describing the key insights found in the rows.
            2. Be concise.
            3. Do not repeat raw numbers unless important.
            4. Show metrics as percentages.
        """,
    output_key="final_summary"
)

In [4]:
insight_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="test_bigquery_agent",
    tools=[bigquery_toolset, preload_memory, AgentTool(agent=summarizer_agent),],
    output_key = "final_answer",
    instruction=f'''
        Your workflow is:
    
            1. Translate the user question into BigQuery SQL, use CTEs only when multiple steps are required. Otherwise use SELECT statements ONLY.
            2. Call the BigQueryToolset `execute_sql` tool using the 'sql generation rules' below:
                * Use CTEs only when multiple steps are required. Otherwise use SELECT statements ONLY.
                * You MUST generate SQL **exclusively in Google BigQuery Standard SQL**.
                * You MUST use only functions that BigQuery supports natively.
                * Forbidden: STRFTIME, DATE_FORMAT, MySQL, PostgreSQL, SQLite, or Python datetime functions
                * Always query from: `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}` unless explicitly directed otherwise by the user.
                * Never hallucinate column names—only use the schema below.
                * Only SELECT the minimum fields needed.
                * Use clean technical aliases (no natural-language-like column names).
                * If a date window is requested (e.g. “last 3 days”), generate SQL that:
                    - Computes it relative to the MAX(Date) in the table.
                    - Example pattern: Date >= (SELECT DATE_SUB(MAX(Date), INTERVAL 3 DAY) FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`)
                    - Do NOT use functions like DECLARE.
                * When computing metrics (CTR, conversion rate, counts, distinct users, funnels, etc.) compute them explicitly in SQL.
                * Never add comments inside the SQL.
                * SQL MUST be the only thing in the "query" string.
                * If unsure whether a function exists in BigQuery, DO NOT GUESS. Always choose the closest known BigQuery function instead.
                
            3. Wait for the BigQueryToolset `execute_sql` tool to return rows.
            4. Call the SummarizerAgent
            5. Wait for the SummarizerAgent tool to return final_summary.
            6. Respond with the final summary. *IMPORTANT*: Return ONLY the result from SummarizerAgent as the final answer.
                        
        Here's the schema for the `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}` table.
        **TABLE SCHEMA**
                    
            date               DATE    - date of the reported campaign performance 
            campaign           STRING  - marketing campaign  
            campaign_id        STRING  - the id of the marketing campaign  
            category           STRING  - the marketing channel for the campaign  
            impressions        STRING  - the total number of times the campaign message was displayed to the audience  
            mark_spent         STRING  - The total amount (in USD) of campaign budget spent on this campaign on this day  
            clicks             STRING  - Indicates how many, of the audience shown the campaign message, clicked on the call-to-action  
            leads              STRING  - Total count of people who signed-up after clicking on the campaign call-to-action  
            orders             STRING  - Total orders received as a result of the campaign 
            revenue            STRING  - Total sales amount from the orders received

        When asked for metrics or kpis or performance, these are the calculation rules to apply to the data.
        **METRIC RULES**
        These are the key metrics that are important to the user in descending order of importance:
            1. **CTR** - take total sum of "impressions" as the denominator and total sum of "clicks" as numerator in a "clicks"/"impression" CTR calculation.
            2. **Leads** conversion rate - use total 'leads' divided by total 'clicks'.
            3. **Orders** conversion rate - use total 'orders' divided by total 'clicks'.
            4. **Average order value** - use total 'revenue' divided by total 'orders'.
            5. **Cost-per-click** - use total 'mark_spent' divided by total 'clicks'.
            6. **Cost-per-lead** - use total 'mark_spent' divided by total 'leads'.
            7. **Cost-per-order** - use total 'mark_spent' divided by total 'orders'.
            8. **Gross profit or loss** - total revenue minus total mark_spent.
            9. **Return on marketing expense** - (total revenue - total mark_spent) divided by total mark_spent.
                
        These are the rules every result output should follow:
        **RESULT RULES**:
            - ALWAYS RETURN THE RESULT FROM SummarizerAgent
            - Never answer without calling BigQuery first.
            - Never include SQL in the final answer.
            - Always pass the BigQuery result directly to the SummarizerAgent.

        If any tool returns status "error", explain the issue to the user clearly.
        
    ''',
    after_agent_callback=auto_save_to_memory,  # Saves after each turn!
    )

In [5]:
memory_service = (InMemoryMemoryService()) 
session_service = InMemorySessionService()

auto_runner = Runner(
    agent=insight_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
    )

In [6]:
with contextlib.redirect_stderr(supp_warning):
    response = await auto_runner.run_debug(
        f"What's are results across all campaigns for the key metrics in the last 7 days in `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`?", verbose=False)


 ### Created new session: debug_session_id

User > What's are results across all campaigns for the key metrics in the last 7 days in `aidev-479603.sample_perf_data.social_media_metrics`?
test_bigquery_agent > The total marketing spend of $1.64 million generated over 211 million impressions and 165,000 clicks. This resulted in 3,448 leads and 445 orders, with a total revenue of $2.39 million. The campaigns show a positive return on marketing expense, indicating profitability.

Key performance indicators highlight a click-through rate of 0.08%, a leads conversion rate of 2.09%, and an orders conversion rate of 13.05% from leads. The average order value stands at $5,370.34, and the cost per order was $3,694.28. The campaigns are demonstrating a strong return on marketing investment.

*   **CTR:** 0.08%
*   **Leads Conversion Rate:** 2.09%
*   **Orders Conversion Rate (from Leads):** 13.05%
*   **Average Order Value (AOV):** $5,370.34
*   **Return on Marketing Expense (ROME):** 145.37%


In [7]:
with contextlib.redirect_stderr(supp_warning):
    response = await auto_runner.run_debug("Can you repeat your previous output?", verbose=False)


 ### Continue session: debug_session_id

User > Can you repeat your previous output?
test_bigquery_agent > The total marketing spend of $1.64 million generated over 211 million impressions and 165,000 clicks. This resulted in 3,448 leads and 445 orders, with a total revenue of $2.39 million. The campaigns show a positive return on marketing expense, indicating profitability.

Key performance indicators highlight a click-through rate of 0.08%, a leads conversion rate of 2.09%, and an orders conversion rate of 13.05% from leads. The average order value stands at $5,370.34, and the cost per order was $3,694.28. The campaigns are demonstrating a strong return on marketing investment.

*   **CTR:** 0.08%
*   **Leads Conversion Rate:** 2.09%
*   **Orders Conversion Rate (from Leads):** 13.05%
*   **Average Order Value (AOV):** $5,370.34
*   **Return on Marketing Expense (ROME):** 145.37%


In [8]:
with contextlib.redirect_stderr(supp_warning):
    response = await auto_runner.run_debug("Which campaign is most cost-efficient this during the last 7 days?", verbose=False)


 ### Continue session: debug_session_id

User > Which campaign is most cost-efficient this during the last 7 days?
test_bigquery_agent > The most cost-efficient campaign over the last 7 days was `instagram_tier2`. This campaign demonstrated a strong return on investment, with revenue significantly outpacing marketing spend. The campaign's efficiency was primarily determined by its low Cost Per Order (CPO).

Here's a breakdown of the key metrics:

*   **Cost Per Click (CPC):** $2.14
*   **Cost Per Lead (CPL):** $2,863.71
*   **Cost Per Order (CPO):** $2,863.71
*   **Return on Marketing Expense (ROME):** 66.54%
